In [3]:
!pip install langchain langchain_community beautifulsoup4 chromadb sentence-transformers
!pip install -U lxml

  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_community-0.3.27-py3-none-any.whl.metadata (2.9 kB)
  Using cached chromadb-1.0.16-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.4 kB)
  Using cached sentence_transformers-5.1.0-py3-none-any.whl.metadata (16 kB)
  Using cached langchain_core-0.3.74-py3-none-any.whl.metadata (5.8 kB)
  Using cached langchain_text_splitters-0.3.9-py3-none-any.whl.metadata (1.9 kB)
  Using cached langsmith-0.4.13-py3-none-any.whl.metadata (14 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.10.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached python_dotenv-1.1.1-p

In [13]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

In [5]:
PERSIST_DIRECTORY = "chroma_db_python_iniciante"

In [6]:
# --- FASE 3: CONSTRUINDO A CADEIA DE RESPOSTAS (RAG) ---
prompt_template = """
Use o contexto a seguir para responder à pergunta no final.
Se você não sabe a resposta, apenas diga que não sabe, não tente inventar uma resposta.
Responda em Português do Brasil. Lembre-se: você é um assistente de Python para iniciantes, então responda claramente, com diversos exemplos passo a passo.

{context}

Pergunta: {question}
Resposta útil:"""

In [7]:
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


In [11]:
# Mesma configuração de embeddings do notebook original
model_name = "intfloat/multilingual-e5-small"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/tmp/ipykernel_327/838217801.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
# Carrega o vetorstore existente a partir do disco
# IMPORTANTE: não passamos 'from_documents', apenas 'persist_directory'
vectorstore = Chroma(
    embedding_function=embeddings,
    persist_directory=PERSIST_DIRECTORY,
)


/tmp/ipykernel_327/1129934144.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [15]:
print("✅ Vector store carregado de", PERSIST_DIRECTORY)
try:
    # algumas versões expõem ._collection.count()
    print("Total de documentos/chunks (aprox.):", vectorstore._collection.count())
except Exception:
    # fallback: não quebra se a API mudar
    pass


✅ Vector store carregado de chroma_db_python_iniciante
Total de documentos/chunks (aprox.): 9293


In [16]:
# 2) LLM (Ollama) e Cadeia de Consulta (RetrievalQA)
# Use o mesmo modelo que você utilizou antes
llm = Ollama(model="llama3", temperature=0)

/tmp/ipykernel_327/3222895476.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3", temperature=0)


In [17]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

In [18]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",   # pode trocar para 'map_reduce' dependendo do seu caso
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)

In [19]:
print("✅ Cadeia de consulta pronta!")

✅ Cadeia de consulta pronta!


In [20]:
# 3) Função utilitária para perguntar
def perguntar(pergunta: str):
    """Executa a consulta no RAG já carregado e imprime resposta + fontes."""
    resp = qa_chain(pergunta)
    print("\n### Resposta:\n", resp['result'])
    print("\n---\n### Fontes:")
    for i, doc in enumerate(resp.get('source_documents', []), start=1):
        meta = doc.metadata if hasattr(doc, 'metadata') else {}
        origem = meta.get('source') or meta.get('file_path') or str(meta)
        score = meta.get('score')
        print(f"[{i}] {origem}" + (f" -> score: {score}" if score is not None else ""))
    return resp

In [24]:
_ = perguntar("o que é uma classe?")


### Resposta:
 Uma pergunta fundamental!

De acordo com a PEP 318, uma definição de classe define um objeto classe (ou seja, uma classe). Uma definição de classe é uma instrução executável que começa com a palavra-chave `class`, seguida do nome da classe e opcionalmente parâmetros de tipo e herança.

Em Python, todas as classes são elas mesmas instâncias de outras classes. A classe de uma classe é conhecida como metaclasse dessa classe, e a maioria das classes tem a classe `type` como sua metaclasse.

Portanto, podemos definir uma classe como um tipo de objeto que define um conjunto de atributos e métodos que podem ser usados para criar instâncias daquela classe.

---
### Fontes:
[1] data/python-3.13-docs-html/reference/compound_stmts.html
[2] data/python-3.13-docs-html/reference/datamodel.html
[3] data/python-3.13-docs-html/reference/datamodel.html
[4] data/python-3.13-docs-html/reference/datamodel.html
